In [ ]:
import pandas as pd
import numpy as np
from IPython.display import HTML, display

In [ ]:
# Upload de Datasets
# https://drive.google.com/drive/folders/1oncC_yOvZeNGmC_hLJeavdM4MPq3BkNd

# deben estar en la carpeta original_datasets

users_ff = pd.read_csv('original_datasets/users_ff.csv')
users_ss1 = pd.read_csv('original_datasets/users_ss1.csv')
users_ss2 = pd.read_csv('original_datasets/users_ss2.csv')
users_ss3 = pd.read_csv('original_datasets/users_ss3.csv')
users_ts = pd.read_csv('original_datasets/users_ts.csv')
users_humans = pd.read_csv('original_datasets/users_gen.csv')

tweets_ff = pd.read_csv('original_datasets/tweets_ff.csv')
tweets_ss1 = pd.read_csv('original_datasets/tweets_ss1.csv')
tweets_ss2 = pd.read_csv('original_datasets/tweets_ss2.csv')
tweets_ss3 = pd.read_csv('original_datasets/tweets_ss3.csv')
tweets_ts = pd.read_csv('original_datasets/tweets_ts.csv')
tweets_humans = pd.read_csv('original_datasets/tweets_gen.csv')

### Dataset Usuarios

In [ ]:
# Agregar columna para categorizar
users_ss1.loc[:,'Cat'] = 'SocialSpambot'
users_ss2.loc[:,'Cat'] = 'SocialSpambot'
users_ss3.loc[:,'Cat'] = 'SocialSpambot'
users_ts.loc[:,'Cat'] = 'TraditionalSpambot'
users_ff.loc[:,'Cat'] = 'FollowersBot'
users_humans.loc[:,'Cat'] = 'Humans'

users_ss1.loc[:,'bot'] = True
users_ss2.loc[:,'bot'] = True
users_ss3.loc[:,'bot'] = True
users_ts.loc[:,'bot'] = True
users_ff.loc[:,'bot'] = True
users_humans.loc[:,'bot'] = False

# Limpiar columnas innecesarias
users_ss1.drop(columns='test_set_1',inplace=True)
users_ss3.drop(columns='test_set_2',inplace=True)
users_humans.drop(columns=['test_set_1','test_set_2'],inplace=True)

# Concatenar datasets de Usuarios (SocialSpambots & Traditional Spambots)
users = pd.concat([users_ss1,users_ss2,users_ss3,users_ts,users_ff,users_humans],ignore_index=True)

In [ ]:
# Drop Cero Columns
users.drop(columns=['contributors_enabled','follow_request_sent','following','is_translator','notifications',],inplace=True)

In [ ]:
users.info()

In [ ]:
total_columns = []
for x in [users_ss1, users_ss2, users_ss3, users_ts, users_ff, users_humans]:
#     print(x.columns.values)
    total_columns = np.concatenate((total_columns, x.columns.values))
    
total_columns = np.unique(np.array(total_columns))
dfHasColumns = pd.DataFrame( columns = total_columns)
for dfName in ['users_ss1', 'users_ss2', 'users_ss3', 'users_ts', 'users_ff', 'users_humans']:
    dataFrame = globals()[dfName]
    cols = dataFrame.columns.values
    hasCol = [1 if x in(cols) else 0 for x in dfHasColumns.columns.values]
    dfHasColumns.loc[dfName] = hasCol
dfHasColumns  = dfHasColumns.T
dfHasColumns.loc[:,'total'] = dfHasColumns.sum(axis=1)

dfHasColumns.sort_values(by='total').head(7)

#### Conclusiones de esta tabla:
La columna 'crawled_at' no se encuentra en users_ff (Folowers Bot).<br>
Esta columna es importante debido a que nos da una idea de la cantidad de meses que ese bot estuvo activo (y nos permite calcular una vida promedio del bot)
Hay tres opciones:<br>
1 No calcular la cantidad de meses antes que un bot se crea.<br>
2 No utilizar ese dataset.<br>
3 Asumir la fecha updated que esta presente en todos los casos

In [ ]:
display()

for x in users_humans.columns.values:
    display(
        users_humans.loc[:,[x]][users_humans.loc[:,[x]].notnull().values].head(2)
    )

In [ ]:
# Exportar a pkl
# users_spam.to_pickle('users_spam.pkl')

### Dataset Tweets

In [ ]:
# Agregar columna para categorizar
tweets_ss1['Cat'] = 'SocialSpambot'
tweets_ss2['Cat'] = 'SocialSpambot'
tweets_ss3['Cat'] = 'SocialSpambot'
tweets_ts['Cat'] = 'TraditionalSpambot'
tweets_ff['Cat'] = 'FollowersBot'
tweets_humans['Cat'] = 'Humans'
# Concatenar datasets de Usuarios (SocialSpambots & Traditional Spambots)
tweets = pd.concat([tweets_ss1,tweets_ss2,tweets_ss3,tweets_ts,tweets_ff,tweets_humans],ignore_index=True)

In [ ]:
tweets.describe()

In [ ]:
tweets['created_at'].notna().sum()

In [ ]:
total_columns = []
for x in [tweets_ss1, tweets_ss2, tweets_ss3, tweets_ts, tweets_ff, tweets_humans]:
#     print(x.columns.values)
    total_columns = np.concatenate((total_columns, x.columns.values))
    
total_columns = np.unique(np.array(total_columns))
dfHasColumns = pd.DataFrame( columns = total_columns)
for dfName in ['tweets_ss1', 'tweets_ss2', 'tweets_ss3', 'tweets_ts', 'tweets_ff', 'tweets_humans']:
    dataFrame = globals()[dfName]
    cols = dataFrame.columns.values
    hasCol = [1 if x in(cols) else 0 for x in dfHasColumns.columns.values]
    dfHasColumns.loc[dfName] = hasCol
dfHasColumns  = dfHasColumns.T
dfHasColumns.loc[:,'total'] = dfHasColumns.sum(axis=1)

dfHasColumns.sort_values(by='total').head(7)

In [ ]:
print(tweets_ss1.shape)
print(tweets_ss2.shape)
print(tweets_ss3.shape)
print(tweets_ts.shape)
print(tweets_ff.shape)
print(teets_humans.shape)

In [ ]:
# Exportar a pkl
# tweets_spam.to_pickle('tweets_spam.pkl')